In [ ]:
from abc import abstractmethod
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
from pandas import json_normalize
import time
import os

df_INN = pd.read_excel('/Users/dmitry/Desktop/Python/Parser/ИНН_ЛК.xlsx')

for i in range(len(df_INN['ИНН компании'])):
    INN = str(df_INN['ИНН компании'][i])
    if len(INN) < 10:
        INN = '0' + INN
    else:
        INN
    URL_get_company_link = 'https://fedresurs.ru/backend/companies?limit=15&offset=0&code='+ INN + '&isActive=true'
    HEADERS_get_company_link ={
                'Accept': 'application/json, text/plain, */*',
                'Pragma': 'no-cache',
                'Cookie': '_ym_visorc_44970568=w; fedresurscookie=e9cfadf77318175b185adc019fc0761e; _ym_isad=2; _ym_d=1609666419; _ym_uid=1609021879885879532',
                'Referer': 'https://fedresurs.ru/search/entity?code=' + INN,
                'Cache-Control': 'no-cache',
                'Host': 'fedresurs.ru',
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.1 Safari/605.1.15 ',
                'Accept-Language': 'ru',
                'Accept-Encoding': 'gzip, deflate, br',
                'Connection': 'keep-alive'
    }
    r_get_company_link = requests.get(URL_get_company_link, headers=HEADERS_get_company_link)
    link = r_get_company_link.json()['pageData'][0]['guid']

    lst = []
    for i in range(0, 15, 15):
        URL_page_2 = 'https://fedresurs.ru/backend/companies/' + link + '/publications?limit=15&offset=' + str(i) + '&searchCompanyEfrsb=true&searchAmReport=true&searchFirmBankruptMessage=true&searchFirmBankruptMessageWithoutLegalCase=false&searchSfactsMessage=true&searchSroAmMessage=true&searchTradeOrgMessage=true'
        HEADERS_2 ={
                    'Accept': 'application/json, text/plain, */*',
                    'Pragma': 'no-cache',
                    'Cookie': '_ym_visorc_44970568=w; fedresurscookie=e9cfadf77318175b185adc019fc0761e; _ym_isad=2; _ym_d=1609666419; _ym_uid=1609021879885879532',
                    'Referer': 'https://fedresurs.ru/company/' + link,
                    'Cache-Control': 'no-cache',
                    'Host': 'fedresurs.ru',
                    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.1 Safari/605.1.15',
                    'Accept-Language': 'ru',
                    'Accept-Encoding': 'gzip, deflate, br',
                    'Connection': 'keep-alive'
        }
        lst.append(URL_page_2)

    lst_df_start = []
    lst_df_stop = []
    lst_df_change = []

    for j in range(len(lst)):
        r_two = requests.get(lst[j], headers=HEADERS_2)
        link_two = r_two.json()['pageData']

        for i in range(len(link_two)):
            leasing = link_two[i]['guid']
            URL_leasing = 'https://fedresurs.ru/backend/sfactmessages/' + leasing
            HEADERS_leasing = {
                'Accept': 'application/json, text/plain, */*',
                'Pragma': 'no-cache',
                'Cookie': '_ym_visorc_44970568=w; fedresurscookie=e9cfadf77318175b185adc019fc0761e; _ym_isad=2; _ym_d=1609666419; _ym_uid=1609021879885879532',
                'Referer': 'https://fedresurs.ru/sfactmessage/' + leasing,
                'Cache-Control': 'no-cache',
                'Host': 'fedresurs.ru',
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.1 Safari/605.1.15',
                'Accept-Language': 'ru',
                'Accept-Encoding': 'gzip, deflate, br',
                'Connection': 'keep-alive'
            }
            r_leasing = requests.get(URL_leasing, headers=HEADERS_leasing)
            if 'content' in r_leasing.json():
                if r_leasing.json()['messageType'] == 'StopFinancialLeaseContract':
                    stop = r_leasing.json()
                    datePublish = stop['datePublish'][:10]
                    contractNumber = stop['content']['contractNumber']
                    contractDate = stop['content']['contractDate'][:10]
                    stopDate = stop['content']['stopDate'][:10]
                    stopReason = stop['content']['stopReason']
                    l = [x['fullName'] for x in stop['content']['lessorsCompanies']]
                    lessorsCompanies = ','.join(l)
                    IP = [x['inn'] for x in stop['content']['lesseesIndividualEntrepreneurs']]
                    lesseesIndividualEntrepreneurs = ','.join(IP)
                    df_stop = pd.DataFrame({'content.contractNumber': [contractNumber], 'content.contractDate': [contractDate], 'content.endDate': [stopDate],  'content.stopReason':  [stopReason],  'content.lessorsCompanies': [lessorsCompanies], 'content.lesseesIndividualEntrepreneurs': [lesseesIndividualEntrepreneurs], 'datePublish': [datePublish]})
                    lst_df_stop.append(df_stop)
                elif r_leasing.json()['messageType'] == 'FinancialLeaseContract':
                    start = r_leasing.json()
                    datePublish = start['datePublish'][:10]
                    typeName = start['typeName']
                    messageType = start['messageType']
                    startDate = start['content']['startDate'][:10]
                    endDate = start['content']['endDate'][:10]
                    contractNumber = start['content']['contractNumber']
                    contractDate = start['content']['contractDate'][:10]
                    clC = [x['classifierCode'] for x in start['content']['subjects']]
                    classifierCode = ','.join(clC)
                    clN = [x['classifierName'] for x in start['content']['subjects']]
                    classifierName = ','.join(clN)
                    dsc = [x['description'] for x in start['content']['subjects']]
                    description = ','.join(dsc)
                    lC = [x['fullName'] for x in start['content']['lessorsCompanies']]
                    lessorsCompanies = ','.join(lC)
                    lS = [x['fullName'] for x in start['content']['lesseesCompanies']]
                    lesseesCompanies = ','.join(lS)
                    LI = [x['inn'] for x in start['content']['lesseesCompanies']]
                    lesseesCompanies_inn = ','.join(LI)
                    IP = [x['inn'] for x in start['content']['lesseesIndividualEntrepreneurs']]
                    lesseesIndividualEntrepreneurs = ','.join(IP)
                    df_start = pd.DataFrame({'content.contractNumber': [contractNumber], 'content.contractDate': [contractDate], 'content.startDate': [startDate],  'content.endDate':  [endDate],  'content.classifierCode': [classifierCode], 'content.classifierName': [classifierName], 'content.description': [description], 'content.lessorsCompanies': [lessorsCompanies], 'content.lesseesCompanies': [lesseesCompanies], 'content.lesseesCompanies_inn': [lesseesCompanies_inn],  'content.lesseesIndividualEntrepreneurs': [lesseesIndividualEntrepreneurs],'datePublish': [datePublish]})
                    lst_df_start.append(df_start)
                elif r_leasing.json()['messageType'] == 'ChangeFinancialLeaseContract':
                    change = r_leasing.json()
                    datePublish = change['datePublish'][:10]
                    typeName = change['typeName']
                    messageType = change['messageType']
                    startDate = change['content']['startDate'][:10]
                    endDate = change['content']['endDate'][:10]
                    contractNumber = change['content']['contractNumber']
                    contractDate = change['content']['contractDate'][:10]
                    clC = [x['classifierCode'] for x in change['content']['subjects']]
                    classifierCode = ','.join(clC)
                    clN = [x['classifierName'] for x in change['content']['subjects']]
                    classifierName = ','.join(clN)
                    dsc = [x['description'] for x in change['content']['subjects']]
                    description = ','.join(dsc)
                    lC = [x['fullName'] for x in change['content']['lessorsCompanies']]
                    lessorsCompanies = ','.join(lC)
                    lS = [x['fullName'] for x in change['content']['lesseesCompanies']]
                    lesseesCompanies = ','.join(lS)
                    LI = [x['inn'] for x in change['content']['lesseesCompanies']]
                    lesseesCompanies_inn = ','.join(LI)
                    IP = [x['inn'] for x in change['content']['lesseesIndividualEntrepreneurs']]
                    lesseesIndividualEntrepreneurs = ','.join(IP)
                    df_change = pd.DataFrame({'content.contractNumber': [contractNumber], 'content.contractDate': [contractDate], 'content.startDate': [startDate],  'content.endDate':  [endDate],  'content.classifierCode': [classifierCode], 'content.classifierName': [classifierName], 'content.description': [description], 'content.lessorsCompanies': [lessorsCompanies], 'content.lesseesCompanies': [lesseesCompanies], 'content.lesseesCompanies_inn': [lesseesCompanies_inn], 'content.lesseesIndividualEntrepreneurs': [lesseesIndividualEntrepreneurs],'datePublish': [datePublish]})
                    lst_df_change.append(df_change)
                else:
                    pass
            else:
                filename = 'Exception/' + r_leasing.json()['guid'] + '.json'
                with open(filename, 'w') as f:
                    f.write(str(r_leasing.json()))
            time.sleep(5)

    if len(lst_df_start) == 0:
        pass
    else:
        df_start_final = pd.concat(lst_df_start)
        df_start_final.to_excel('/Users/dmitry/Desktop/Python/Parser/START/df_start_final_' + INN + '.xlsx', sheet_name='Старт')

    if len(lst_df_stop) == 0:
        pass
    else:
        df_stop_final = pd.concat(lst_df_stop)
        df_stop_final.to_excel('/Users/dmitry/Desktop/Python/Parser/END/df_stop_final_' + INN + '.xlsx', sheet_name='Cтоп')

    if len(lst_df_change) == 0:
        pass
    else:
        df_change_final = pd.concat(lst_df_change)
        df_change_final.to_excel('/Users/dmitry/Desktop/Python/Parser/CHANGE/df_change_final_' + INN + '.xlsx', sheet_name='Изменение')
    print(INN)

fold = ['START', 'CHANGE', 'END']

for i in range(len(fold)):
    directory = '/Users/dmitry/Desktop/Python/Parser/' + fold[i]
    files = os.listdir(directory)
    print(files)

    lst_s = []
    lst_c = []
    lst_e = []
    for j in range(len(files)):
        if files[j] == '.DS_Store':
            pass
        else:
            df = pd.read_excel('/Users/dmitry/Desktop/Python/Parser/' + fold[i] + '/' + files[j])
            if fold[i] == 'START':
                lst_s.append(df)
                df_s = pd.concat(lst_s)
                df_s.to_excel('/Users/dmitry/Desktop/Python/Parser/df_s.xlsx')
            elif fold[i] == 'CHANGE':
                lst_c.append(df)
                df_c = pd.concat(lst_c)
                df_c.to_excel('/Users/dmitry/Desktop/Python/Parser/df_c.xlsx')
            else:
                lst_e.append(df)
                df_e = pd.concat(lst_e)
                df_e.to_excel('/Users/dmitry/Desktop/Python/Parser/df_e.xlsx')

df = df_s.merge(df_c, how='outer', right_on='content.contractNumber', left_on='content.contractNumber')
df = df.merge(df_e, how='outer', right_on='content.contractNumber', left_on='content.contractNumber')
df.to_excel('/Users/dmitry/Desktop/Python/Parser/df.xlsx')